In [2]:
from gurobipy import *
import pandas as pd
import numpy as np
from pandas import DataFrame

In [3]:
def mySorter(tuple1,tuple2):
    n_tup1 = (tuple1[0],tuple2[0])
    n_tup2 = (tuple1[1],tuple2[1])
    return [n_tup1,n_tup2]

def var2name(arr):
    l=[]
    for val in DataFrame(arr).unstack():
        l.append(val.varName)
    name_array = np.array(l)
    name_array = name_array.reshape(arr.shape)
    return np.transpose(name_array)

try:
    
    m = Model("Part_2")
    
    #Read data
    flows = np.array(pd.read_csv('~/FacilityDesign_SameArea/dep5_inst1/flows.txt', header=None))
    unit_cost = np.array(pd.read_csv('~/FacilityDesign_SameArea/dep5_inst1/unit_cost.txt', header=None))
    
    #Initialize values
    x = flows.shape #get the shape of the array (e.g 3,3 array)
    numDepts=x[0]
    numLocs = x[1]
    
    #Create variables
    obj_vars = []    #objective function variables
    obj_expr = 0
    
    for i in range(1,numDepts+1):
        
        for j in range(1,numLocs+1):
            v = "x"+str(i)+"_"+str(j)
            v = m.addVar(vtype=GRB.BINARY, name=v)
            obj_vars.append(v)
            
    #Update variables
    m.update()
    
    obj_arr = np.array(obj_vars)
    obj_arr = obj_arr.reshape(flows.shape)
    
    for k in range(len(obj_arr)):
        for l in range(len(obj_arr)):
            temp = obj_arr
            temp = temp[k+1:,]
            temp = np.delete(temp,(l),axis=1)
            for var in DataFrame(temp).unstack():
                tup1 = (k,l)
                tup2 = np.where(var2name(obj_arr)==var.varName)
                flow_tup,cost_tup = mySorter(tup1,tup2)
                obj_expr += obj_arr[k,l]*var*flows[flow_tup]*unit_cost[cost_tup]
                
    #Set Objective
    m.setObjective(obj_expr,GRB.MINIMIZE)
            
    #Constraints
    for n in range(flows.shape[0]):
        m.addConstr(np.sum(obj_arr[n])==1,"Dept"+str(n+1))
        m.addConstr(np.sum(obj_arr[:,n])==1,"Loc"+str(n+1))
    
    #Optimize
    m.optimize()
    
    for value in m.getVars():
        if value.x != 0:
            print value.varName, value.x
    
    print "Objective: ", m.objVal
    
except GurobiError:
    print 'Error. Solution Not Found'

Optimize a model with 10 rows, 25 columns and 50 nonzeros
Model has 200 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [0e+00, 0e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 1e+00]
Found heuristic solution: objective 2658
Modified 25 Q diagonals
Presolve time: 0.01s
Presolved: 10 rows, 25 columns, 50 nonzeros
Presolved model has 225 quadratic objective terms
Variable types: 0 continuous, 25 integer (25 binary)

Root relaxation: objective -2.631758e+03, 36 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -2631.7580    0   25 2658.00000 -2631.7580   199%     -    0s
H    0     0                    2516.0000000 -2631.7580   205%     -    0s
H    0     0                    2274.0000000 -2631.7580   216%     -    0s
     0     2 -2631.7580    0   25 2274.00000 -2631.7580   216%   

In [5]:
obj_expr

<gurobi.QuadExpr: 0.0 + [ [ 5.] x1_1 * x2_2 + [ 4.] x1_1 * x3_2 + [ 5.] x1_1 * x2_3 + [ 4.] x1_1 * x3_3 + [ 5.] x1_2 * x2_1 + [ 4.] x1_2 * x3_1 + [ 10.] x1_2 * x2_3 + [ 8.] x1_2 * x3_3 + [ 5.] x1_3 * x2_1 + [ 4.] x1_3 * x3_1 + [ 10.] x1_3 * x2_2 + [ 8.] x1_3 * x3_2 + [ 6.] x2_1 * x3_2 + [ 6.] x2_1 * x3_3 + [ 6.] x2_2 * x3_1 + [ 12.] x2_2 * x3_3 + [ 6.] x2_3 * x3_1 + [ 12.] x2_3 * x3_2 ]>

In [177]:
np.where(var2name(obj_arr)==var.varName)

(array([1]), array([2]))

In [192]:
def var2name(arr):
    l=[]
    for val in DataFrame(arr).unstack():
        l.append(val.varName)
    name_array = np.array(l)
    name_array = name_array.reshape(arr.shape)
    return np.transpose(name_array)

In [193]:
var2name(obj_arr)

array([['x1_1', 'x1_2', 'x1_3'],
       ['x2_1', 'x2_2', 'x2_3'],
       ['x3_1', 'x3_2', 'x3_3']], 
      dtype='|S4')